 My to-do list:
- Diagram training process
- Add filters for bad chips (in distribution tails at levels that look like nonsense) for training -- do this once in preprocessing of "cleaned xBD" directory
- TTS model from baseline for damage
- Keras manual training loop in Jupyter
- Use CPU for inference
- Maybe recode baseline in chainer if I can figure out how to transfer the weights, so I can work in a single package and single process
- detect bad chips
- SR 30 end to end 100%
- 1 example for each disaster and damage type 100%
- threshold size for localization to reduce   fragments
- break out all scores per chip to spot hard cases systematically
- use image display liberally on all steps in process flow for sanity check

I will try making the smallest set of images that contains one of each incident and per incident one of each damage level, then see if I can overtrain my model individually to match grou d truth for each of these images.   This is to verify sizing and capacity.  And shake out bugs like the 1 pixel chips.  Then worry about training and then take all the money.  The plan should always end with take all the money, not some ego preserving excuse about self betterment.

https://towardsdatascience.com/keras-custom-training-loop-59ce779d60fb

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [10]:
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict
from sklearn.model_selection import train_test_split
logging.basicConfig(level=logging.INFO)

# Configurations
NUM_WORKERS = 4
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 120
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_STEP = 150

damage_intensity_encoding = defaultdict(lambda: 0)
damage_intensity_encoding['destroyed'] = 3
damage_intensity_encoding['major-damage'] = 2
damage_intensity_encoding['minor-damage'] = 1
damage_intensity_encoding['no-damage'] = 0

def process_img(img_array, polygon_pts, scale_pct):
    """Process Raw Data into

            Args:
                img_array (numpy array): numpy representation of image.
                polygon_pts (array): corners of the building polygon.

            Returns:
                numpy array: .

    """

    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
    xmin = max(int(xmin - (xdiff * scale_pct)), 0)
    xmax = min(int(xmax + (xdiff * scale_pct)), width)
    ymin = max(int(ymin - (ydiff * scale_pct)), 0)
    ymax = min(int(ymax + (ydiff * scale_pct)), height)

    (X,Y,Z)=img_array.shape
    
    if not(0 <= xmin < X and 0 < xmax <= X and 0 <= ymin < Y and 0 < ymax <= Y):
        print(f'X: xmin {xmin}, xmax {xmax}, X {X}; Y: ymin {ymin}, ymax {ymax}, Y {Y}')
        assert 0 <= xmin < X and 0 < xmax <= X and 0 <= ymin < Y and 0 < ymax <= Y
    
    return img_array[ymin:ymax, xmin:xmax, :]

input_path='/home/catskills/Desktop/dataxv2/xBD'

x_data = []
y_data = []

disasters = [folder for folder in os.listdir(input_path) if not folder.startswith('.')]
disaster_paths = ([input_path + "/" +  d + "/images" for d in disasters])
image_paths = []
image_paths.extend([(disaster_path + "/" + pic) for pic in os.listdir(disaster_path)] for disaster_path in disaster_paths)
img_paths = np.concatenate(image_paths)

image_info=[]

for img_path in img_paths:
    img_obj = Image.open(img_path)
    img_array = np.array(img_obj)
    label_path = img_path.replace('png', 'json').replace('images', 'labels')
    label_file = open(label_path)
    label_data = json.load(label_file)
    for feat in label_data['features']['xy']:
        try:
            damage_type = feat['properties']['subtype']
        except: # pre-disaster damage is default no-damage
            damage_type = "no-damage"
            continue
        polygon_geom = shapely.wkt.loads(feat['wkt'])
        polygon_pts = np.array(list(polygon_geom.exterior.coords))
        poly_img = process_img(img_array, polygon_pts, 0.8)
        image_info.append((poly_img.shape, img_path, feat['properties']['uid'], poly_img))
    if img_path=='/home/catskills/Desktop/dataxv2/xBD/mexico-earthquake/images/mexico-earthquake_00000168_post_disaster.png':
        break

In [11]:
oneA = [x for x in image_info if x[0][0] ==1]
oneB = [x for x in image_info if x[0][1] ==1]
big=[x for x in image_info if x[0][0]*x[0][1]>1000**2]

print(len(oneA),len(oneB),len(big))

0 0 1


In [12]:
plt.imshow(oneA[0][-1])
plt.title(oneA[0][1]);

IndexError: list index out of range

In [ ]:
plt.imshow(oneB[0][-1])
plt.title(oneB[0][1]);

In [ ]:

len(big)

In [ ]:
plt.imshow(big[0][-1])
plt.title(big[0][1]);

In [ ]:
[(x[0:2],y,z) for x,y,z,w in oneB]

In [ ]:
[(x[0:2],y,z) for x,y,z,w in big]

In [ ]:
img_path=oneA[0][1]

In [ ]:
img_path

In [ ]:
feat['properties']['uid']

In [ ]:
feat['wkt']

In [ ]:
    img_info=[]
    img_obj = Image.open(img_path)
    img_array = np.array(img_obj)
    label_path = img_path.replace('png', 'json').replace('images', 'labels')
    label_file = open(label_path)
    label_data = json.load(label_file)
    for feat in label_data['features']['xy']:
        try:
            damage_type = feat['properties']['subtype']
        except: # pre-disaster damage is default no-damage
            damage_type = "no-damage"
            continue
        polygon_geom = shapely.wkt.loads(feat['wkt'])
        polygon_pts = np.array(list(polygon_geom.exterior.coords))
        poly_img = process_img(img_array, polygon_pts, 0.8)
        img_info.append((poly_img.shape, img_path, feat['properties']['uid'], poly_img))
        if poly_img.shape[0]==1:
            break

In [ ]:
plt.imshow(poly_img)

In [ ]:
feat['wkt']

In [ ]:
polygon_geom

In [ ]:
polygon_pts

In [ ]:
plt.plot(polygon_pts[:,0], polygon_pts[:,1])

In [ ]:
matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from skimage import io
from glob import glob
fns=glob('/home/catskills/Desktop/dataxv2/classifier/*.png')
dimensions=set([io.imread(fn).shape[0:2] for fn in fns])
len(dimensions)

d=list(dimensions)
X=np.array([x for x,y in dimensions])
Y=np.array([y for x,y in dimensions])

X.min(), X.max(), Y.min(), Y.max()

[(x,y) for (x,y) in dimensions if x==1 or y==1]M

S=list(sorted([np.sqrt(x*y) for x,y in dimensions]))
plt.hist(S,bins=100);MM

dimensions=[(fn,)+io.imread(fn).shape[0:2] for fn in fns]
[fn for fn,x,y in dimensions if x>1000 and y>1000]

bigs=[fn for fn,x,y in dimensions if x>1000 and y>1000]

plt.imshow(io.imread(bigs[0]))